In [7]:
# Importing all the required files. Note that if you are trying to run my Arduino code, you have to first install
# the Arduino IDE, and go to Files --> Examples --> Firmata, and upload onto your Arduino UNO.
# Note that if you have a copy of the Arduino UNO, like I did, then you will need to install this driver:
# http://www.wch.cn/download/CH341SER_MAC_ZIP.html , trust me its not a virus XD
# After that follow the instructions, then go to terminal, and write ls /dev/tty* to get the port number your arduino
# is connected on, and put it in the board = Arduino() code below.
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import pyfirmata
# from pyfirmata import Arduino, util
# import time

%config Completer.use_jedi = False

In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [9]:
# Upload the haarcascasde xml. Can be downloaded from GitHub. Also, upload the model downloaded from the last line of the code on collab.
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
mask_detection = tf.keras.models.load_model("mask_new_detection.h5")

In [10]:
# Set Up text, fonts, scales etc
text_mask = "Mask On"
text_no_mask = "Mask Off"

font = cv2.FONT_HERSHEY_SIMPLEX
scale = 0.8

In [11]:
# Use CV2 to convert our image into gray scale. This is for haarcascade,
def predict(image):  
  face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_frame = cv2.resize(face_frame, (224, 224))
  face_frame = img_to_array(face_frame)
  face_frame = np.expand_dims(face_frame, axis=0)
  face_frame = preprocess_input(face_frame)
  prediction = mask_detection.predict(face_frame)

  return prediction[0][0]

In [12]:
# Setup the rectangles that would appear depending on the probabs
def detector(gray_image, frame):
#     board = Arduino('/dev/tty.usbmodem14201')
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5)
    
    for (x,y,w,h) in faces:

        roi_color = frame[y:y+h, x:x+w]
        mask = predict(roi_color)

        
        if mask > 0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, text =text_mask, org =(x+50,y-10), fontFace =font, fontScale = scale, color =(0,255,0),
            thickness = 2)
            
#             for x in range(int("1")):
#                 board.digital[13].write(1)
#                 time.sleep(1)
#                 board.digital[13].write(0)
#                 time.sleep(1)

        
        elif mask<=0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
            cv2.putText(frame, text =text_no_mask, org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),
            thickness = 2)
            
#             for x in range(int("1")):
#                 board.digital[12].write(1)
#                 time.sleep(1)
#                 board.digital[12].write(0)
#                 time.sleep(1)
            
    return frame



In [13]:
# And finally, capture video, and detect whether wearing mask or not. Hit q to quit the program BTW.
video_cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    detect = detector(gray_frame, frame) 
    
    cv2.imshow("Video", detect)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
video_cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 76ms/step
